# Build GPT from scratch

taken from Video from Andrej Karpathy (https://www.youtube.com/watch?v=kCc8FmEb1nY)

We use the tiny_shapespeare.txt data.

References are:
* https://arxiv.org/pdf/1706.03762.pdf


In [4]:
with  open('data/shakespeare/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [5]:
print('length of dataset: ', len(text))

length of dataset:  1115394


In [6]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [7]:
# vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
# create mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello World!"))
print(decode(encode("Hello World!")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Hello World!


Alternate tokinizers:
* SentencePiece: https://github.com/google/sentencepiece
* TikToken (used by openAI): https://github.com/openai/tiktoken

In [9]:
!pip install tiktoken

Keyring is skipped due to an exception: org.freedesktop.DBus.Error.InvalidFileContent: D-Bus library appears to be incorrectly set up: see the manual page for dbus-uuidgen to correct this issue. (Failed to open "/var/lib/dbus/machine-id": No such file or directory; UUID file '/etc/machine-id' should contain a hex string of length 32, not length 0, with no other text)
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tiktoken (from versions: none)
ERROR: No matching distribution found for tiktoken


In [10]:
import tiktoken

enc = tiktoken.get_encoding('gpt2')
print(enc.n_vocab)

print(enc.encode("Hello World!"))

print(enc.decode(enc.encode("Hello World!")))

ModuleNotFoundError: No module named 'tiktoken'

In [11]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [12]:
# split dataset into train and validation (90/10)

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [13]:
block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
x = train_data[:block_size]
y = train_data[1:block_size+1] 
for t in range(block_size):
    ctx = x[:t+1]
    target = y[t]
    print(ctx, target)

tensor([18]) tensor(47)
tensor([18, 47]) tensor(56)
tensor([18, 47, 56]) tensor(57)
tensor([18, 47, 56, 57]) tensor(58)
tensor([18, 47, 56, 57, 58]) tensor(1)
tensor([18, 47, 56, 57, 58,  1]) tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)


In [15]:
torch.manual_seed(1337)

batch_size = 4 # number of independent sequences processed in parallel 
block_size = 8 # context length

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x  = torch.stack([data[i:i+block_size] for i in ix])
    y  = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

print('inputs:  ', xb.shape, xb)
print('targets: ', yb.shape, yb)
print('--')

for b in range(batch_size):
    for t in range(block_size):
        ctx = xb[b,:t+1]
        target = yb[b, t]
        print(ctx, target)


inputs:   torch.Size([4, 8]) tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:  torch.Size([4, 8]) tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--
tensor([24]) tensor(43)
tensor([24, 43]) tensor(58)
tensor([24, 43, 58]) tensor(5)
tensor([24, 43, 58,  5]) tensor(57)
tensor([24, 43, 58,  5, 57]) tensor(1)
tensor([24, 43, 58,  5, 57,  1]) tensor(46)
tensor([24, 43, 58,  5, 57,  1, 46]) tensor(43)
tensor([24, 43, 58,  5, 57,  1, 46, 43]) tensor(39)
tensor([44]) tensor(53)
tensor([44, 53]) tensor(56)
tensor([44, 53, 56]) tensor(1)
tensor([44, 53, 56,  1]) tensor(58)
tensor([44, 53, 56,  1, 58]) tensor(46)
tensor([44, 53, 56,  1, 58, 46]) tensor(39)
tensor([44, 53, 56,  1, 58, 46, 39]) tensor(58)
tensor([44, 53, 56,  1, 58, 46, 39, 58]) tensor(

In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reas off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_token):
        # idx is (B, T) array of indices in the ctx
        for _ in range(max_new_token):
            # get predictions    
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)

logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_token=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [17]:
# create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [18]:
batch_size=32

for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate th eloss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.382369041442871


In [19]:
print(decode(m.generate(idx, max_new_token=500)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


In [20]:
# towards self-attention
torch.manual_seed(1337)

B,T,C = 4,8,32
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [21]:
# we want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,c)
        xbow[b,t] = torch.mean(xprev,0)

In [22]:
x[0]
xbow[0]

tensor([[ 1.8077e-01, -6.9988e-02, -3.5962e-01, -9.1520e-01,  6.2577e-01,
          2.5510e-02,  9.5451e-01,  6.4349e-02,  3.6115e-01,  1.1679e+00,
         -1.3499e+00, -5.1018e-01,  2.3596e-01, -2.3978e-01, -9.2111e-01,
          1.5433e+00,  1.3488e+00, -1.3964e-01,  2.8580e-01,  9.6512e-01,
         -2.0371e+00,  4.9314e-01,  1.4870e+00,  5.9103e-01,  1.2603e-01,
         -1.5627e+00, -1.1601e+00, -3.3484e-01,  4.4777e-01, -8.0164e-01,
          1.5236e+00,  2.5086e+00],
        [-2.4116e-01, -1.6063e-01,  3.2526e-01, -3.9683e-01,  3.9208e-01,
          5.7976e-01, -9.9692e-02, -1.1702e-01, -7.3193e-02,  1.2198e-01,
         -4.0159e-01, -1.0025e+00, -4.8488e-01,  1.6602e-01, -7.5923e-01,
          4.2481e-01,  1.4972e+00, -4.7132e-01,  8.1860e-01,  3.4460e-01,
         -1.7740e+00,  1.2990e+00,  2.1250e+00, -5.7775e-01,  7.8881e-01,
         -1.5365e+00, -1.6947e-01, -2.7318e-01,  6.1334e-01,  3.6581e-01,
          1.5667e+00,  1.0527e+00],
        [-4.3893e-01,  9.2179e-02,  1.99

In [ ]:
head_size = 16
key = nn.Linear(C, head_size, bias= False)
query = nn.Linear(C, head_size, bias= False)
value = nn.Linear(C, head_size, bias= False)
k = key(x)   # (B,T, 16)
q = query(x) # (B,T, 16)
wei = q @ k.transpose(-2, -1) # (B,T,16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T,T))

wei  = wei.masked_fill(tril ==0, float('-inf'))

wei  = F.softmax(wei, dim=-1)
Z= wei @ v

In [24]:
tril = torch.tril(torch.ones(T,T))
#wei  = torch.zeros((T,T))
wei  = wei.masked_fill(tril ==0, float('-inf'))
wei  = F.softmax(wei, dim=-1)

v = value(x)

out = wei @ x
out = wei @ v

In [25]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [26]:
out[0]

tensor([[-0.1571,  0.8801,  0.1615, -0.7824, -0.1429,  0.7468,  0.1007, -0.5239,
         -0.8873,  0.1907,  0.1762, -0.5943, -0.4812, -0.4860,  0.2862,  0.5710],
        [ 0.6764, -0.5477, -0.2478,  0.3143, -0.1280, -0.2952, -0.4296, -0.1089,
         -0.0493,  0.7268,  0.7130, -0.1164,  0.3266,  0.3431, -0.0710,  1.2716],
        [ 0.4823, -0.1069, -0.4055,  0.1770,  0.1581, -0.1697,  0.0162,  0.0215,
         -0.2490, -0.3773,  0.2787,  0.1629, -0.2895, -0.0676, -0.1416,  1.2194],
        [ 0.1971,  0.2856, -0.1303, -0.2655,  0.0668,  0.1954,  0.0281, -0.2451,
         -0.4647,  0.0693,  0.1528, -0.2032, -0.2479, -0.1621,  0.1947,  0.7678],
        [ 0.2510,  0.7346,  0.5939,  0.2516,  0.2606,  0.7582,  0.5595,  0.3539,
         -0.5934, -1.0807, -0.3111, -0.2781, -0.9054,  0.1318, -0.1382,  0.6371],
        [ 0.3428,  0.4960,  0.4725,  0.3028,  0.1844,  0.5814,  0.3824,  0.2952,
         -0.4897, -0.7705, -0.1172, -0.2541, -0.6892,  0.1979, -0.1513,  0.7666],
        [ 0.1866, -0.0

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)


n_embd= 32


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))


    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class BigramLanguageModel2(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reas off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # idx and targets are both (B,T) tensor of integers
        tok_embed = self.token_embedding_table(idx) # (B,T,C)
        pos_embed = self.position_embedding_table(torch.arange(T)) # (T,C)
        x= tok_embed + pos_embed
        x=self.sa_head(x)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_token):
        # idx is (B, T) array of indices in the ctx
        for _ in range(max_new_token):
            idx_conds = idx[:, -block_size:]
            # get predictions    
            logits, loss = self(idx_conds)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLanguageModel2()

logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_token=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.1999, grad_fn=<NllLossBackward0>)

ee'nCXLFs?Z,gfrx.dJ$tRFCEun U3pmPej3Pt-y?'SwrWl!$K-:I,IwIR$&,?cNIUN'wONtErKcfjmKZvF,GvpzGweVoHSrQJs,


In [28]:
# create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size=32

for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate th eloss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.3033900260925293


In [29]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_token=100)[0].tolist()))


Turl-chee
'A Pout;
I myoum, yousshou tand hererord oche ty Xas faur ciuprue:
Hot as thaillgout mandd


In [30]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        #self.proj = nn.Linear(head_size * num_heads, n_embd)
        #self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        #out = self.dropout(self.proj(out))
        return out

class BigramLanguageModel3(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reas off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = MultiHeadAttention(4, n_embd//4)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # idx and targets are both (B,T) tensor of integers
        tok_embed = self.token_embedding_table(idx) # (B,T,C)
        pos_embed = self.position_embedding_table(torch.arange(T)) # (T,C)
        x= tok_embed + pos_embed
        x=self.sa_head(x)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_token):
        # idx is (B, T) array of indices in the ctx
        for _ in range(max_new_token):
            idx_conds = idx[:, -block_size:]
            # get predictions    
            logits, loss = self(idx_conds)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLanguageModel3()

logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_token=100)[0].tolist()))    

torch.Size([256, 65])
tensor(4.2469, grad_fn=<NllLossBackward0>)

VG,vRG:ylxdtbnHA;,ea$Pc,qO$kX!Iky!Xf;,!jDC$EmZ'TSsjdBszJ& -EFOHcfMxUc$QnkJXuWT,$cjTy'kSB,&rK$MNw,Lpm


In [31]:
# create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size=32

for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate th eloss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.0846638679504395


In [32]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_token=500)[0].tolist()))    


QUEN, I toone soun bouet Sell sent, hee suge.

Ither anltt ther dearcathe Yihestor heighh ou houldee.

Nombleraciove:
Ed To Jo-shacerut as herers dien thore! wichave beaposss.
O: my pin:
Aard:
Hy whout Es DO tho:
Por chow thruo inn faltat deth wirds kf
OuF Ber may lastish, let donty my tas so thoud su orthou sto gral halll eand hould Bilen I,
USeck Huls useiferp?
For thertu.'
Fa gapu of, beegt kin'd sho isild try, loster moll,
El:
Wingulde, lence hircand the lorrd he will color-sp
he as
If the I


In [33]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            #nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class BigramLanguageModel4(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reas off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = MultiHeadAttention(4, n_embd//4)
        self.fwd = FeedFoward(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # idx and targets are both (B,T) tensor of integers
        tok_embed = self.token_embedding_table(idx) # (B,T,C)
        pos_embed = self.position_embedding_table(torch.arange(T)) # (T,C)
        x = tok_embed + pos_embed
        x = self.sa_head(x)
        x = self.fwd(x)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_token):
        # idx is (B, T) array of indices in the ctx
        for _ in range(max_new_token):
            idx_conds = idx[:, -block_size:]
            # get predictions    
            logits, loss = self(idx_conds)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLanguageModel4()



In [34]:
# create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size=32

for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate th eloss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.111990451812744


In [35]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_token=500)[0].tolist())) 


Ther spe; no ade: hate a for to?
SO:
O what i
Is chais,
out then be no botal for miess, wicie,
As me preaday:
Bye
Tho prost thak have,
Four uparpon by me, do frome yest,
Now's poted, and you care.

BUCHESTELLO:
Eld; I camjark par,
Secay, I and his liabe in dowo,
Fir;
Wo:
My pred yest;
And shall
I wich thatthe cet prown:
To of wout the you, misper:
Unrhat this mitp dotly! the comink cim nat I dreto as fie yir,
Buntle.
OF IID:
Vead,
'Is but fraw motiesseris is is ther it leark.

UCKE:
Tit freartav


In [36]:
n_layer = 6
n_head = 6
n_embd = 384

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel4(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reas off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        # idx and targets are both (B,T) tensor of integers
        tok_embed = self.token_embedding_table(idx) # (B,T,C)
        pos_embed = self.position_embedding_table(torch.arange(T)) # (T,C)
        x = tok_embed + pos_embed # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_token):
        # idx is (B, T) array of indices in the ctx
        for _ in range(max_new_token):
            idx_conds = idx[:, -block_size:]
            # get predictions    
            logits, loss = self(idx_conds)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLanguageModel4()


In [ ]:
# create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate th eloss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

In [ ]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_token=500)[0].tolist())) 